[View in Colaboratory](https://colab.research.google.com/github/Curiousss/InkerIntern/blob/master/Copy_of_DNST_CIFAR10_AUG.ipynb)

In [0]:
# https://keras.io/
!pip install -q keras
import keras

Using TensorFlow backend.


In [0]:
import keras
from keras.datasets import cifar10
from keras.models import Model, Sequential
from keras.layers import Dense, Dropout, Flatten, Input, AveragePooling2D, merge, Activation
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras.layers import Concatenate
from keras.optimizers import Adam

In [0]:
# this part will prevent tensorflow to allocate all the avaliable GPU Memory
# backend
import tensorflow as tf
from keras import backend as k

# Don't pre-allocate memory; allocate as-needed
config = tf.ConfigProto()
config.gpu_options.allow_growth = True

# Create a session with the above options specified.
k.tensorflow_backend.set_session(tf.Session(config=config))

In [0]:
# Hyperparameters
batch_size = 128
num_classes = 10
epochs = 50
l = 20
num_filter = 20
compression = 0.8
dropout_rate = 0.2

In [0]:
# Load CIFAR10 Data
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
img_height, img_width, channel = x_train.shape[1],x_train.shape[2],x_train.shape[3]

# convert to one hot encoing 
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

In [0]:
'''
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
datagen = ImageDataGenerator(
        rotation_range=40,
        horizontal_flip=True)
        vertical_flip=True,
        horizontal_flip=True
     fill_mode='nearest',
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
train_gen = datagen.flow(x_train, y_train, batch_size)
test_gen = datagen.flow(x_test, y_test, batch_size)
'''

"\nfrom keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img\ndatagen = ImageDataGenerator(\n        rotation_range=40,\n        horizontal_flip=True)\n        vertical_flip=True,\n        horizontal_flip=True\n     fill_mode='nearest',\n        width_shift_range=0.2,\n        height_shift_range=0.2,\n        shear_range=0.2,\n        zoom_range=0.2,\ntrain_gen = datagen.flow(x_train, y_train, batch_size)\ntest_gen = datagen.flow(x_test, y_test, batch_size)\n"

In [0]:
# Dense Block
def add_denseblock(input, num_filter = 12, dropout_rate = 0.2):
    global compression
    temp = input
    for _ in range(l):
        BatchNorm = BatchNormalization()(temp)
        relu = Activation('relu')(BatchNorm)
        Conv2D_3_3 = Conv2D(int(num_filter*compression), (3,3), use_bias=False ,padding='same')(relu)
        if dropout_rate>0:
          Conv2D_3_3 = Dropout(dropout_rate)(Conv2D_3_3)
        concat = Concatenate(axis=-1)([temp,Conv2D_3_3])
        
        temp = concat
        
    return temp

In [0]:
def add_transition(input, num_filter = 12, dropout_rate = 0.2):
    global compression
    BatchNorm = BatchNormalization()(input)
    relu = Activation('relu')(BatchNorm)
    Conv2D_BottleNeck = Conv2D(int(num_filter*compression), (1,1), use_bias=False ,padding='same')(relu)
    if dropout_rate>0:
      Conv2D_BottleNeck = Dropout(dropout_rate)(Conv2D_BottleNeck)
    avg = AveragePooling2D(pool_size=(2,2))(Conv2D_BottleNeck)
    
    return avg

In [0]:
def output_layer(input):
    global compression
    BatchNorm = BatchNormalization()(input)
    relu = Activation('relu')(BatchNorm)
    AvgPooling = AveragePooling2D(pool_size=(2,2))(relu)
    flat = Flatten()(AvgPooling)
    output = Dense(num_classes, activation='softmax')(flat)
    
    return output

In [0]:
#num_filter = 12
#dropout_rate = 0.2
#l = 12
input = Input(shape=(img_height, img_width, channel,))
First_Conv2D = Conv2D(num_filter, (3,3), use_bias=False ,padding='same')(input)

First_Block = add_denseblock(First_Conv2D, num_filter, dropout_rate)
First_Transition = add_transition(First_Block, num_filter, dropout_rate)

Second_Block = add_denseblock(First_Transition, num_filter, dropout_rate)
Second_Transition = add_transition(Second_Block, num_filter, dropout_rate)

Third_Block = add_denseblock(Second_Transition, num_filter, dropout_rate)
Third_Transition = add_transition(Third_Block, num_filter, dropout_rate)

Last_Block = add_denseblock(Third_Transition,  num_filter, dropout_rate)
output = output_layer(Last_Block)


In [0]:
model = Model(inputs=[input], outputs=[output])
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
conv2d_45 (Conv2D)              (None, 32, 32, 20)   540         input_2[0][0]                    
__________________________________________________________________________________________________
batch_normalization_45 (BatchNo (None, 32, 32, 20)   80          conv2d_45[0][0]                  
__________________________________________________________________________________________________
activation_45 (Activation)      (None, 32, 32, 20)   0           batch_normalization_45[0][0]     
__________________________________________________________________________________________________
conv2d_46 


__________________________________________________________________________________________________
activation_97 (Activation)      (None, 8, 8, 176)    0           batch_normalization_97[0][0]     
__________________________________________________________________________________________________
conv2d_98 (Conv2D)              (None, 8, 8, 16)     25344       activation_97[0][0]              
__________________________________________________________________________________________________
dropout_96 (Dropout)            (None, 8, 8, 16)     0           conv2d_98[0][0]                  
__________________________________________________________________________________________________
concatenate_91 (Concatenate)    (None, 8, 8, 192)    0           concatenate_90[0][0]             
                                                                 dropout_96[0][0]                 
__________________________________________________________________________________________________
batch_nor

In [0]:
# determine Loss function and Optimizer
model.compile(loss='categorical_crossentropy',
              optimizer=Adam(),
              metrics=['accuracy'])

In [0]:
from keras.callbacks import ModelCheckpoint
#filepath= “weightsbest.hdf5”
#checkpoint = ModelCheckpoint(filepath, monitor=‘val_acc’, verbose=1, save_best_only=True, mode=‘max’)
filepath="weights-improvement-{epoch:02d}-{val_acc:.2f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]

In [0]:
'''model.fit_generator(train_gen,
                    epochs=epochs,
                    verbose=1,
                    steps_per_epoch=len(x_train) / batch_size,
                    validation_data=test_gen)
                    '''
model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(x_test, y_test),
         callbacks=callbacks_list)

Train on 50000 samples, validate on 10000 samples
Epoch 1/50
50000/50000 [==============================] - 656s 13ms/step - loss: 1.4700 - acc: 0.4637 - val_loss: 3.0693 - val_acc: 0.3639

Epoch 00001: val_acc improved from -inf to 0.36390, saving model to weights-improvement-01-0.36.hdf5
Epoch 2/50
13824/50000 [=======>......................] - ETA: 7:03 - loss: 1.1099 - acc: 0.5976

50000/50000 [==============================] - 628s 13ms/step - loss: 1.0451 - acc: 0.6200 - val_loss: 1.4637 - val_acc: 0.5695

Epoch 00002: val_acc improved from 0.36390 to 0.56950, saving model to weights-improvement-02-0.57.hdf5
Epoch 3/50
29312/50000 [================>.............] - ETA: 4:01 - loss: 0.9039 - acc: 0.6758

50000/50000 [==============================] - 628s 13ms/step - loss: 0.8820 - acc: 0.6851 - val_loss: 1.7329 - val_acc: 0.5497

Epoch 00003: val_acc did not improve from 0.56950
Epoch 4/50
39936/50000 [======================>.......] - ETA: 1:57 - loss: 0.7766 - acc: 0.7254

50000/50000 [==============================] - 627s 13ms/step - loss: 0.7692 - acc: 0.7277 - val_loss: 1.0596 - val_acc: 0.6785

Epoch 00004: val_acc improved from 0.56950 to 0.67850, saving model to weights-improvement-04-0.68.hdf5
Epoch 5/50
35968/50000 [====================>.........] - ETA: 2:43 - loss: 0.6881 - acc: 0.7590

50000/50000 [==============================] - 627s 13ms/step - loss: 0.6845 - acc: 0.7600 - val_loss: 1.8609 - val_acc: 0.5924

Epoch 00005: val_acc did not improve from 0.67850
Epoch 6/50
41984/50000 [========================>.....] - ETA: 1:33 - loss: 0.6290 - acc: 0.7791

...

50000/50000 [==============================] - 625s 13ms/step - loss: 0.0929 - acc: 0.9670 - val_loss: 0.7703 - val_acc: 0.8487

Epoch 00046: val_acc did not improve from 0.87480
Epoch 47/50
44416/50000 [=========================>....] - ETA: 1:05 - loss: 0.0915 - acc: 0.9675

50000/50000 [==============================] - 625s 13ms/step - loss: 0.0916 - acc: 0.9677 - val_loss: 0.6121 - val_acc: 0.8799

Epoch 00047: val_acc improved from 0.87480 to 0.87990, saving model to weights-improvement-47-0.88.hdf5
Epoch 48/50
36992/50000 [=====================>........] - ETA: 2:31 - loss: 0.0859 - acc: 0.9693

50000/50000 [==============================] - 626s 13ms/step - loss: 0.0861 - acc: 0.9694 - val_loss: 0.7080 - val_acc: 0.8492

Epoch 00048: val_acc did not improve from 0.87990
Epoch 49/50
42112/50000 [========================>.....] - ETA: 1:31 - loss: 0.0847 - acc: 0.9697

50000/50000 [==============================] - 626s 13ms/step - loss: 0.0862 - acc: 0.9691 - val_loss: 0.6734 - val_acc: 0.8667

Epoch 00049: val_acc did not improve from 0.87990
Epoch 50/50
43776/50000 [=========================>....] - ETA: 1:12 - loss: 0.0817 - acc: 0.9708

50000/50000 [==============================] - 625s 13ms/step - loss: 0.0818 - acc: 0.9707 - val_loss: 0.7804 - val_acc: 0.8557

Epoch 00050: val_acc did not improve from 0.87990


In [0]:
model.load_weights("weights-improvement-47-0.88.hdf5")
model.summary()

In [0]:
# Test the model
score = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

10000/10000 [==============================] - 54s 5ms/step
Test loss: 0.6121428798764944
Test accuracy: 0.8799


In [0]:
# Save the trained weights in to .h5 format
model.save_weights("DNST_model.h5")
print("Saved model to disk")

Saved model to disk


In [0]:
from google.colab import files

files.download('DNST_model.h5')